In [1]:
import cv2  
import os
import numpy as np 
import face_recognition
import pickle
import cvzone
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db
import requests
from datetime import datetime
from firebase_admin import storage

In [6]:



cap=cv2.VideoCapture(0)
cap.set(3,640)
cap.set(4,488)

background=cv2.imread('Resources/background.png')       #matrix (numpy)
folder_mode_path="Resources/Modes"
mode_pathlist = os.listdir(folder_mode_path)
img_mode_list=[]
for path in mode_pathlist:
    img_mode_list.append(cv2.imread(os.path.join(folder_mode_path,path)))


# load the encoding file
print("Loading encode file.....")
file=open('EncodeFile.p','rb')
encodeListKnownwithID=pickle.load(file)
file.close()
encodeListKnown,studentID=encodeListKnownwithID

modeType=0
counter=0
imgStudent=[]
id=-1

while True:
    success,image=cap.read()

    imgs=cv2.resize(image,(0,0),None,0.25,0.25)
    imgs=cv2.cvtColor(imgs,cv2.COLOR_BGR2RGB)

    faceCurFrame=face_recognition.face_locations(imgs)
    encodeCurFrame=face_recognition.face_encodings(imgs,faceCurFrame)

    background[162:162+480,55:55+640]=image
    background[44:44+633,808:808+414]=img_mode_list[modeType]

    if faceCurFrame:

        for encodeface , faceloc in zip(encodeCurFrame,faceCurFrame):
            matches=face_recognition.compare_faces(encodeListKnown,encodeface)
            faceDistance=face_recognition.face_distance(encodeListKnown,encodeface)
            #print("matches", matches)
            #print("faceDis", faceDistance)

            matchIndex=np.argmin(faceDistance)
            # print("Match Index", matchIndex)
            if matches[matchIndex]:
                #print("Known Face Detected")
                #print(studentID[matchIndex])
                y1,x2,y2,x1=faceloc
                y1,x2,y2,x1=y1*4 ,x2*4 , y2*4 ,x1*4
                bbox=55+x1 ,162+y1 , x2-x1 , y2-y1
                background=cvzone.cornerRect(background,bbox,rt=0)
                id=studentID[matchIndex]
                if counter == 0:
                        cvzone.putTextRect(background, "Loading", (275, 400))
                        cv2.imshow("Face Attendance", background)
                        cv2.waitKey(1)
                        counter = 1
                        modeType = 1


        if counter !=0:

            if counter==1:
                studentInfo = db.reference(f'students/{id}').get()
                print(studentInfo)

                # Get the Image from the storage
                img_url = studentInfo['profile_picture']
                #array = np.frombuffer(img_url.download_as_string(), np.uint8)
                #imgStudent = cv2.imdecode(array, cv2.COLOR_BGRA2BGR) 
                imgStudent = cv2.imread(img_url)
                # Update data of attendance
                datetimeObject=datetime.strptime(studentInfo['last_attendance_time'],"%Y-%m-%d %H:%M:%S")
                secondsElapsed=(datetime.now()-datetimeObject).total_seconds()
                print(secondsElapsed)

                if secondsElapsed >30:
                    ref=db.reference(f'students/{id}')
                    studentInfo['total_attendance']+=1
                    ref.child('total_attendance').set(studentInfo['total_attendance'])
                    ref.child('last_attendance_time').set(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
                
                else:
                    modeType=3
                    counter=0
                    background[44:44 + 633, 808:808 + 414] =img_mode_list[modeType]
        
            if modeType !=3:

                if 10<counter<20:
                    modeType=2
                background[44:44 + 633, 808:808 + 414] = img_mode_list[modeType]

                if counter <=10:
                    cv2.putText(background, str(studentInfo['total_attendance']), (861, 125),
                                    cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 0), 2)
                    cv2.putText(background, str(studentInfo['major']), (1006, 550),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
                    cv2.putText(background, str(id), (1006, 493),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
                    cv2.putText(background, str(studentInfo['standing']), (910, 625),
                                    cv2.FONT_HERSHEY_COMPLEX, 1, (100, 100, 100), 2)
                    cv2.putText(background, str(studentInfo['year']), (1025, 625),
                                    cv2.FONT_HERSHEY_COMPLEX, 1, (100, 100, 100), 2)
                    cv2.putText(background, str(studentInfo['starting_year']), (1125, 625),
                                    cv2.FONT_HERSHEY_COMPLEX, 0.6, (100, 100, 100), 2)

                    (w, h), _ = cv2.getTextSize(studentInfo['Name'], cv2.FONT_HERSHEY_COMPLEX, 1, 1)
                    offset = (414 - w) // 2
                    cv2.putText(background, str(studentInfo['Name']), (808 + offset, 445),
                                    cv2.FONT_HERSHEY_COMPLEX, 1, (50, 50, 50), 1)
            
                    background[175:175 + 216, 909:909 + 216] = imgStudent
            
                counter +=1

                if counter >=20:
                    counter=0
                    modeType=0
                    studentInfo=[]
                    imgStudent=[]
                    background[44:44 + 633, 808:808 + 414] = img_mode_list[modeType]

    else:
        modeType=0
        counter=0
        

    cv2.imshow("webcamp",image)
    cv2.imshow("Face Attendance",background)
    key=cv2.waitKey(10)
    if key==27:
        break
cap.release()
cv2.destroyAllWindows()

Loading encode file.....
{'Name': 'Gehad ahmed', 'last_attendance_time': '2025-01-29 19:27:50', 'major': 'AI', 'profile_picture': 'Images\\2100723.jpeg', 'standing': 'A', 'starting_year': 2024, 'total_attendance': 19, 'year': 4}
486.976832
{'Name': 'Gehad ahmed', 'last_attendance_time': '2025-01-29 19:35:57', 'major': 'AI', 'profile_picture': 'Images\\2100723.jpeg', 'standing': 'A', 'starting_year': 2024, 'total_attendance': 20, 'year': 4}
14.892484
{'Name': 'Gehad ahmed', 'last_attendance_time': '2025-01-29 19:35:57', 'major': 'AI', 'profile_picture': 'Images\\2100723.jpeg', 'standing': 'A', 'starting_year': 2024, 'total_attendance': 20, 'year': 4}
15.941371
{'Name': 'Gehad ahmed', 'last_attendance_time': '2025-01-29 19:35:57', 'major': 'AI', 'profile_picture': 'Images\\2100723.jpeg', 'standing': 'A', 'starting_year': 2024, 'total_attendance': 20, 'year': 4}
17.460904
{'Name': 'Gehad ahmed', 'last_attendance_time': '2025-01-29 19:35:57', 'major': 'AI', 'profile_picture': 'Images\\2100